In [0]:
import sys
import os
import pandas as pd
import numpy as np
import warnings
import time
import matplotlib.pyplot as plt
import numpy.random as npr
import random
randseed = 123
print("random seed: ", randseed)
random.seed(randseed)
np.random.seed(randseed)
from google.colab import files


In [0]:
src = list(files.upload().values())[0]
open('CEVAE_FUNCTIONS.py','wb').write(src)
from CEVAE_FUNCTIONS import DATA, Evaluator, get_y0_y1, init_qz,p_x_z, p_t_z, p_y_zt, q_t_x, q_y_xt, q_z_tyx

In [0]:
src = list(files.upload().values())[0]
open('CEVAE.py','wb').write(src)
import CEVAE as cevae 

In [0]:
#zip with 3 files
files.upload()

In [0]:
!unzip snp_simulated1_8.zip
!mkdir -p data_s
!mv snp_simulated1_8.txt data_s/
!mv snp_simulated1_truecauses.txt data_s/
!mv snp_simulated1_y01.txt data_s/
!mkdir -p results


In [0]:
version = 8
start = 0
nsim = 5000
letter = 'a'

pd.set_option('display.max_columns', 500)

from scipy.stats import ttest_ind,ttest_rel

n_units = 5000
n_causes = 10000# 10% var


start_time = time.time()
train_path = 'data_s//snp_simulated1_'+str(version)+'.txt'
y01_path = 'data_s//snp_simulated1_y01.txt'
y01 = np.asmatrix(pd.read_pickle(y01_path))[:,version]
at0 = []
at1 = []
cate = []
y01_predicted =[]
y01_ = []


In [0]:
for (y0, y1, y10, y01_pred, yte) in cevae.model(n_causes,train_path,y01,start,version,nsim):
    at0.append(y0)
    at1.append(y1)
    cate.append(y10)
    y01_predicted.append(y01_pred)
    y01_.append(yte)
    if len(at0)%200 == 0:
        output = pd.DataFrame({'at0':at0,
                               'at1':at1,
                               'cate':cate})

        predictions = np.stack( y01_predicted, axis=0 ).reshape(len(y01_predicted),len(y01_predicted[0]))
        testing_set = np.stack( y01_, axis=0 ).reshape(len(y01_),len(y01_[0]))
        predictions = pd.DataFrame(np.transpose(predictions))
        testing_set = pd.DataFrame(np.transpose(testing_set))
        output.to_pickle('results//cevae_output_sim'+str(version)+'_'+letter+'.txt')
        predictions.to_pickle('results//cevae_pred'+str(version)+'_'+letter+'.txt')
        testing_set.to_pickle('results//cevae_test'+str(version)+'_'+letter+'.txt')


output = pd.DataFrame({'at0':at0,'at1':at1,'cate':cate})

predictions = np.stack( y01_predicted, axis=0 ).reshape(len(y01_predicted),len(y01_predicted[0]))
testing_set = np.stack( y01_, axis=0 ).reshape(len(y01_),len(y01_[0]))
predictions = pd.DataFrame(np.transpose(predictions))
testing_set = pd.DataFrame(np.transpose(testing_set))

#_2 : from 0-5000
output.to_pickle('results//cevae_output_sim'+str(version)+'_'+letter+'.txt')
predictions.to_pickle('results//cevae_pred'+str(version)+'_'+letter+'.txt')
testing_set.to_pickle('results//cevae_test'+str(version)+'_'+letter+'.txt')
print("--- %s minutes ---" % str((time.time() - start_time)/60))

!zip -r results_sim5_7000_9999.zip results/
